In [ ]:
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D
from keras.models import Sequential
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# test augmentation on one image
test_image = cv2.imread('./temp/train/no_doctor/D423.png')

writer = ImageDataGenerator(rescale=1./255,
                            rotation_range=20,
                            fill_mode='constant',
                            cval=125, # grey
                            height_shift_range=0.2,
                            horizontal_flip=True,
                            vertical_flip=True)

imagewriter = writer.flow(np.expand_dims(test_image, axis=0),
                            batch_size=1,
                            save_to_dir = '.',
                            save_prefix="",
                            save_format="png")
write_total = 0
for image in imagewriter:
    write_total += 1
    if write_total == 10:
        break

In [ ]:
# if test is ok, apply augmentation to whole dataset
# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255,
                                   height_shift_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True)
                                   
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        temp_folder / 'train',  # This is the source directory for training images
        target_size=(256, 256),  # All images will be resized to 256 x 256
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        temp_folder / 'test',
        target_size=(256, 256),
        batch_size=20,
        class_mode='binary')

In [ ]:
def mole_model():
 
    model = Sequential()
    model.add(Conv2D(32, (5, 5), strides=(1, 1), padding='same', input_shape=(256,256,3), activation='relu'))
    model.add(Dropout(0.25))
 
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', activation='relu'))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3, 3), strides=(2, 2), padding='same', activation='relu')) # Layer 4
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compile model 
    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

mole_model_a = mole_model()

In [ ]:
es = EarlyStopping(monitor='val_loss',
                   mode='min',
                   verbose=1,
                   patience=4,
                   restore_best_weights=True)

mc = ModelCheckpoint('best_model.h5',
                     monitor='val_accuracy',
                     mode='max',
                     save_best_only=True)

history = mole_model_a.fit(train_generator,
                           epochs=1000,
                           validation_data=validation_generator,
                           batch_size = 20,
                           callbacks = [es, mc])

In [ ]:
mole_model_a.save_weights("model.h5")